In [1]:
import pandas as pd
import csv
import tqdm
from collections import defaultdict

In [2]:
def get_lines_csv(file, delimiter) -> list:
    """
    Read a file and return the lines in a list
    :param delimiter:
    :param file:
    :return: List of lines
    """
    lines = []
    with open(file, "r", encoding="utf-8") as f:
        csv_reader = csv.reader(f, delimiter=delimiter)
        csv.field_size_limit(100100000000)
        for j, row in tqdm.tqdm(enumerate(csv_reader), desc="Getting lines of " + file):
            lines.append(row)
    return lines

def get_headers_index(header, file, sep="\t"):
    if header == "category_name":
        return get_mapping_column_index(file, sep)["category_name"]
    if header == "Labels":
        print(get_mapping_column_index(file, sep))
        return get_mapping_column_index(file, sep)["Labels"]
    if header == "Material":
        return get_mapping_column_index(file, sep)["Material"]
    if header == "search_price":
        return get_mapping_column_index(file, sep)["search_price"]
    if header == "rrp_price":
        return get_mapping_column_index(file, sep)["rrp_price"]
    if header == "delivery_cost":
        return get_mapping_column_index(file, sep)["delivery_cost"]
    if header == "Title":
        return get_mapping_column_index(file, sep)["Title"]
    if header == "Fashion:suitable_for":
        return get_mapping_column_index(file, sep)["Fashion:suitable_for"]
    if header == "merchant_name":
        return get_mapping_column_index(file, sep)[header]
    if header == "aw_deep_link":
        return get_mapping_column_index(file, sep)["aw_deep_link"]
    if header == "Title":
        return get_mapping_column_index(file, sep)["Title"]
    if header == "description":
        return get_mapping_column_index(file, sep)["description"]
    if header == "merchant_product_id":
        return get_mapping_column_index(file, sep)["merchant_product_id"]
    if header == "colour":
        return get_mapping_column_index(file, sep)["colour"]
    if header == "aw_deep_link":
        return get_mapping_column_index(file, sep)["aw_deep_link"]
    if header == "item_id":
        return get_mapping_column_index(file, sep)["item_id"]
    if header == "category_normalized":
        return get_mapping_column_index(file, sep)["category_normalized"]
    if header == "color_normalized":
        return get_mapping_column_index(file, sep)["color_normalized"]
    if header == "label_0":
        return get_mapping_column_index(file, sep)["label_0"]
    if header == "label_1":
        return get_mapping_column_index(file, sep)["label_1"]
    if header == "label_2":
        return get_mapping_column_index(file, sep)["label_2"]
    if header == "label_3":
        return get_mapping_column_index(file, sep)["label_3"]
    if header == "color_normalized_0":
        return get_mapping_column_index(file,sep)["color_normalized_0"]
    if header == "color_normalized_1":
        return get_mapping_column_index(file,sep)["color_normalized_1"]
    if header == "color_normalized_2":
        return get_mapping_column_index(file,sep)["color_normalized_2"]
    if header =="saison":
        return get_mapping_column_index(file,sep)["saison"]
    if header =="saison_conf_score_index":
        return get_mapping_column_index(file,sep)["saison_conf_score_index"]
    else:
        return get_mapping_column_index(file, sep)[header]
def get_mapping_column_index(file, delimiter) -> dict:
    """
    Create the mapping columnName: Index
    :param delimiter:
    :param file:
    :return: dictionary: columnName: index
    """
    mapping = {}
    with open(file, "r", encoding="utf-8") as f:
        csv_reader = csv.reader(f, delimiter=delimiter)
        for row in csv_reader:
            mapping = {columnName: index for index, columnName in enumerate(row)}
            break
    return mapping

In [3]:
data_path = "/home/graphn/repositories/you_conscious/data_processing/data_working_directory/filtered/2021-02-18_filtered_only_matching_categories_datafeed.csv"
list_articles = get_lines_csv(data_path, "\t")

Getting lines of /home/graphn/repositories/you_conscious/data_processing/data_working_directory/filtered/2021-02-18_filtered_only_matching_categories_datafeed.csv: 120207it [00:04, 26227.62it/s]


In [4]:
list_articles = list_articles[1:]

In [5]:
ranking_merchant_ean = {
    "Avocadostore":0,
    "i'm walking":1,
    "mirapodo":2,
    "OTTO":3,
}

In [6]:
def replace_merchant_names_ean_order(merchant_name):
    if merchant_name == "Avocadostore":
        return "0"+merchant_name
    if merchant_name == "Im walking":
        return "1"+merchant_name
    if merchant_name == "mirapodo":
        return "2"+merchant_name
    if merchant_name == "OTTO":
        return "3"+merchant_name
    else:
        return merchant_name


In [7]:
def ean_cleanser( list_articles: list) -> list:
    # {ean:(list_index,merchant)}
    # todo first case with over 2k
    # if list only avocado store keep all
    #diese EAN gibts bei avocadostore und mirapodo 889556801404
    list_clean_ean_articles = []
    list_index_articles_cleansed = []
    list_articles_to_return = []
    articles_index_to_return = []
    merchant_ean_to_clean = ["0Avocadostore", "1Im walking", "2mirapodo", "3OTTO"]
    ean_mapping = defaultdict(list)
    ean_relevant_merchant = (ranking_merchant_ean.keys())
    ean_index = get_headers_index("ean", data_path)
    merchant_name_index = get_headers_index("merchant_name", data_path)
    for a, article in enumerate(list_articles):
        ean = article[ean_index]
        ean_mapping[ean].append({"merchant_name": replace_merchant_names_ean_order(article[merchant_name_index]),
                                 "index": a})
        list_index_articles_cleansed.append(a)

    for ean, merchants in ean_mapping.items():
        ean_cleansed = False
        if ean == str(889556801404):
            print("xx",merchants)
        length_merchant_names = len(merchants)
        list_merchant_names = [merchant["merchant_name"] for merchant in merchants]
        set_merchant_names = set(list_merchant_names)
        if ean != "":
            if len(set(list_merchant_names).intersection(merchant_ean_to_clean)) > 0:
                # print(set_merchant_names)
                if "0Avocadostore" in list_merchant_names:
                    if len(set_merchant_names) == 1:
                        for merchant in merchants:
                            articles_index_to_return.append(merchant["index"])
                    else:
                        for merchant in merchants:
                            if merchant["merchant_name"] == "0Avocadostore":
                                articles_index_to_return.append(merchant["index"])



                else:
                    ordered_merchants = []
                    ord_merchants = {}
                    ordered_merchants = [merchant["merchant_name"]+"-"+str(merchant["index"]) for merchant in merchants]
                    merchant_to_return = ordered_merchants[0]
                    index_to_return = merchant_to_return.split("-")[-1]
                    index_to_return = int(index_to_return)
                    articles_index_to_return.append(index_to_return)




        else:
            for merchant in merchants:
                articles_index_to_return.append(merchant["index"])
    for article_index in articles_index_to_return:
        if type(article_index) == int:
            list_articles_to_return.append(list_articles[article_index])
    return list_articles_to_return

In [8]:
ec = ean_cleanser(list_articles)
print(len(list_articles),len(ec))

xx [{'merchant_name': '0Avocadostore', 'index': 113012}, {'merchant_name': '0Avocadostore', 'index': 113013}]
120206 77292


In [9]:
def write_2_file(list_articles, output_file, delimiter: str = "\t"):
    """
    write a list to a csv file
    :param delimiter:
    :param output_file:
    :param list_articles:
    """
    with open(output_file, "w", encoding="utf-8", newline="") as o:
        csv_writer = csv.writer(o, delimiter=delimiter)
        for element in list_articles:
            if element is not None:
                csv_writer.writerow(element)

In [10]:
write_2_file(ec,"18.csv")